<a href="https://colab.research.google.com/github/hmtrii/tirg/blob/main/Compute_recall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# INSTALL

In [2]:
!git clone https://github.com/hmtrii/tirg.git

Cloning into 'tirg'...
remote: Enumerating objects: 103, done.
remote: Counting objects: 100% (103/103), done.
remote: Compressing objects: 100% (72/72), done.
remote: Total 103 (delta 55), reused 71 (delta 29), pack-reused 0
Receiving objects: 100% (103/103), 3.30 MiB | 17.14 MiB/s, done.
Resolving deltas: 100% (55/55), done.


In [3]:
!rm -rf /content/Fashion200k
!mkdir /content/Fashion200k
!cp -r /content/drive/MyDrive/TIRG/dataset/fashion-200k/labels /content/Fashion200k
!cp /content/drive/MyDrive/TIRG/dataset/test_queries.txt /content/Fashion200k
!mkdir /content/Fashion200k/women

In [4]:
!tar -xf /content/drive/MyDrive/TIRG/dataset/fashion-200k/women.tar.gz -C /content/Fashion200k/women

In [5]:
!pip install tensorboardX

     |████████████████████████████████| 317kB 8.0MB/s 


In [6]:
!pip install torch==1.2.0 torchvision==0.4.0    

     |████████████████████████████████| 748.9MB 18kB/s 
     |████████████████████████████████| 8.8MB 10.5MB/s 
  Found existing installation: torch 1.7.0+cu101
    Uninstalling torch-1.7.0+cu101:
      Successfully uninstalled torch-1.7.0+cu101
  Found existing installation: torchvision 0.8.1+cu101
    Uninstalling torchvision-0.8.1+cu101:
      Successfully uninstalled torchvision-0.8.1+cu101


In [7]:
!pip install Pillow==5.2.0

     |████████████████████████████████| 2.0MB 9.0MB/s 
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: Pillow 7.0.0
    Uninstalling Pillow-7.0.0:
      Successfully uninstalled Pillow-7.0.0


# LOAD DATASET

In [1]:
cd /content/tirg

/content/tirg


In [2]:
from tqdm import tqdm
import torch
import numpy as np
from tools import pkl
import time
from test_retrieval import test
from main import load_dataset, create_model_and_optimizer
from LSH import create_hash_table

In [3]:
class Opt:
    def __init__(self):
        self.dataset = "fashion200k"
        self.dataset_path = "/content/Fashion200k"
        self.batch_size = 32
        self.embed_dim = 512
        self.hashing = True
        self.retrieve_by_random = True
        self.model = "tirg"
        self.learning_rate = 1e-2
        self.weight_decay = 1e-6
        self.pretrained_model = "/content/drive/MyDrive/TIRG/runs/exp2/latest_checkpoint.pth"

In [4]:
opt = Opt()
trainset, testset = load_dataset(opt)
model, _ = create_model_and_optimizer(opt, [t for t in trainset.get_all_texts()])
model.eval()

Reading dataset  fashion200k
read dress_train_detect_all.txt
read jacket_train_detect_all.txt
read pants_train_detect_all.txt
read top_train_detect_all.txt
read skirt_train_detect_all.txt
Fashion200k: 172049 images
53099 unique captions
Modifiable images 106464
read top_test_detect_all.txt
read jacket_test_detect_all.txt
read skirt_test_detect_all.txt
read dress_test_detect_all.txt
read pants_test_detect_all.txt
Fashion200k: 29789 images
trainset size: 172049
testset size: 29789
Creating model and optimizer for tirg


Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/checkpoints/resnet18-5c106cde.pth
100%|██████████| 44.7M/44.7M [00:07<00:00, 6.15MB/s]


Load checkpoint from /content/drive/MyDrive/TIRG/runs/exp2/latest_checkpoint.pth


TIRG(
  (normalization_layer): NormalizationLayer()
  (soft_triplet_loss): TripletLoss()
  (img_model): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=Fal

Compute all images feature

In [ ]:
# all_imgs = []
# imgs = []
# for i in tqdm(range(len(testset.imgs))):
#     imgs += [testset.get_img(i)]
#     if len(imgs) >= opt.batch_size or i == len(testset.imgs) - 1:
#         if 'torch' not in str(type(imgs[0])):
#             imgs = [torch.from_numpy(d).float() for d in imgs]
#         imgs = torch.stack(imgs).float()

#         if torch.cuda.is_available():
#             imgs = torch.autograd.Variable(imgs).cuda()
#         else:
#             imgs = torch.autograd.Variable(imgs).cpu()

#         imgs = model.extract_img_feature(imgs).data.cpu().numpy()
#         all_imgs += [imgs]
#         imgs = []
# all_imgs = np.concatenate(all_imgs)
# all_captions = [img['captions'][0] for img in testset.imgs]

100%|██████████| 29789/29789 [04:36<00:00, 107.54it/s]


Compute all queries feature

In [ ]:
# imgs = []
# mods = []
# all_queries = []
# test_queries = testset.get_test_queries()
# for t in tqdm(test_queries):
#     imgs += [testset.get_img(t['source_img_id'])]
#     mods += [t['mod']['str']]
#     if len(imgs) >= opt.batch_size or t is test_queries[-1]:
#         if 'torch' not in str(type(imgs[0])):
#             imgs = [torch.from_numpy(d).float() for d in imgs]
#         imgs = torch.stack(imgs).float()

#         if torch.cuda.is_available():
#             imgs = torch.autograd.Variable(imgs).cuda()
#         else:
#             imgs = torch.autograd.Variable(imgs).cpu()

#         mods = [t for t in mods]
#         f = model.compose_img_text(imgs, mods).data.cpu().numpy()
#         all_queries += [f]
#         imgs = []
#         mods = []
# all_queries = np.concatenate(all_queries)
# all_target_captions = [t['target_caption'] for t in test_queries]

  0%|          | 17/33480 [00:00<03:19, 167.58it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1350: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 33480/33480 [04:20<00:00, 128.45it/s]


In [5]:
test_queries = testset.get_test_queries()
all_queries = pkl.pkl_load("/content/drive/MyDrive/TIRG/pkl_ver2/all_norm_queries_feature.pkl")
all_target_captions = pkl.pkl_load("/content/drive/MyDrive/TIRG/pkl_ver2/all_target_captions.pkl")
all_imgs = pkl.pkl_load("/content/drive/MyDrive/TIRG/pkl_ver2/all_norm_imgs_feature.pkl")
all_captions = pkl.pkl_load("/content/drive/MyDrive/TIRG/pkl_ver2/all_captions.pkl")

In [6]:
# Create random queries
np.random.seed(4)
id_random = np.random.choice(all_queries.shape[0], 5000)
queries_feature = []
for id in id_random:
    queries_feature += [all_queries[id]]

queries_feature = np.array(queries_feature)

# Normal

In [10]:
tic = time.time()
sims = queries_feature.dot(all_imgs.T)
for i, t in enumerate(test_queries):
	try:
		sims[i, t["source_img_id"]] = -10e10  # remove query image
	except:
		pass

nn_result = [np.argsort(-sims[i, :])[:110] for i in range(sims.shape[0])]
out = []
nn_result = [[all_captions[nn] for nn in nns] for nns in nn_result]
# for k in [1, 5, 10, 50, 100]:
r = 0.0
# for i, nns in enumerate(nn_result):
for id, nns in zip(id_random, nn_result):
    if all_target_captions[id] in nns[:100]:
        r += 1
r /= len(nn_result)
out += [('recall_top' + str(k) + '_correct_composition', 100)]
tac = time.time()
print(out)
print(f"{round(tac - tic, 3)}s")

[('recall_top100_correct_composition', 100)]
13.362s


# Hashing

In [11]:
def compute_similary_by_hasing(all_imgs_feature, queries_feature, test_queries):
	hash_table = create_hash_table(all_imgs_feature, 12, 512)
	sims = []
	for i, query in enumerate(queries_feature):
		idx_imgs = hash_table.__getitem__(query)
		spatial_search = np.array([all_imgs_feature[i] for i in idx_imgs])
		if spatial_search.shape[0] != 0:
			dis_value = query.dot(spatial_search.T)
			sim = np.full(all_imgs_feature.shape[0], -10e10)
			for i in idx_imgs:
				sim[i] = dis_value[0]
				dis_value = dis_value[1:]
		else:
			sim = query.dot(all_imgs_feature.T)

		sim[test_queries[i]['source_img_id']] = -10e10
		sims += [sim]

	return np.array(sims)

In [12]:
tic = time.time()
sims = compute_similary_by_hasing(all_imgs, queries_feature, test_queries)
nn_result = [np.argsort(-sims[i, :])[:100] for i in range(sims.shape[0])]
out = []
nn_result = [[all_captions[nn] for nn in nns] for nns in nn_result]
# for k in [1, 5, 10, 50, 100]:
r = 0.0
# for i, nns in enumerate(nn_result):
for id, nns in zip(id_random, nn_result):
    if all_target_captions[id] in nns[:100]:
        r += 1
r /= len(nn_result)
out += [('recall_top' + str(100) + '_correct_composition', r)]
tac = time.time()
print(out)
print(f"{round(tac - tic, 3)}s")

[('recall_top100_correct_composition', 0.2656)]
11.093s
